In [ ]:
from IPython.core.debugger import set_trace
for x in range(0, 3):
    set_trace()
    print(x)

Part 1 -
1a) w1 = -2.04427331  w2 = 3.99683416 b = -0.92427055
1b) Levenberg–Marquardt as per implementation below

In [55]:
#%%
import os
import numpy as np
import scipy.io
import scipy.optimize as optimization
#from IPython.core.debugger import set_trace

# Data
filedir = os.path.dirname(os.path.realpath('__file__'))
datapath = os.path.join(filedir,'assign1_data.mat')
data = scipy.io.loadmat(datapath)

# fitter wants shape (k,M) (k number of predictors)
# data['x'] has shape (M,k) so we fix that
print(data['x'].shape)
x1 = data['x'][:,0]
x2 = data['x'][:,1]
X = np.array([x1,x2])
# argg - shape (100,1) - gives obstuse error in the fitter
# we fix that too (flatten)
y = np.array(data['y']).flatten()

# Initial guess
w0 = np.array([1, 1, 1])

# Objective function
# y = w1 * x1 + w2 * x2 + b.
def func(X, b, w1, w2):
  # unpack independent vars
  rval = w1*X[0] + w2*X[1] + b
  return rval

result = optimization.curve_fit(func, X, y, w0)
print(result[0])

(100, 2)
[-0.92427055 -2.04427331  3.99683416]


Part 2

In [58]:
import os
import numpy as np
import scipy.io
import scipy.optimize as optimization
#from IPython.core.debugger import set_trace

EPS = 0.001
N_EPOCH = 1000
def dw(w,d,X):
    # using numpy einstein summation to vectorize the computation
    # vector of coefficients (d-w.x) for each pattern (alpha)     
    C = d - np.einsum('i,ij->j', w, X) # shape is (100,)
    # C_i * X_ji or C * transpose(X)
    dw = EPS*np.einsum('i,ji->j', C, X) # shape is (3,)
    return dw

def main():
    # data
    filedir = os.path.dirname(os.path.realpath('__file__'))
    datapath = os.path.join(filedir,'assign1_data.mat')
    data = scipy.io.loadmat(datapath)
    x1 = data['x'][:,0]
    x2 = data['x'][:,1]
    # inputs - with bias input tied high, shape is (3,100)
    X = np.array([np.full(len(x1),1),x1,x2])
    # output
    d = np.array(data['y']).flatten()
    # starting vector of weights
    w = [1,1,1]

    for _ in range(N_EPOCH):
        w = w + dw(w,d,X)
    print(w)

# invoke main
main()

[-0.92233149 -2.0427036   3.99155586]
